<a href="https://colab.research.google.com/github/cchighman/ImageAI-YOLOv3-Fine-Tuning-Vehicle-Classification/blob/master/ImageAI_Fine_Tuning_Resnet50_for_Vehicle_Classification_based_on_YOLOV3_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install imageai
%tensorflow_version 1.x

     |████████████████████████████████| 184kB 4.7MB/s 
TensorFlow 1.x selected.


Tranfer Learning from Previous Prius Detection Model -- specializing against traffic camera images

In [8]:
import os
from imageai.Prediction.Custom import ModelTraining

model_trainer = ModelTraining()
model_trainer.setModelTypeAsResNet()
model_trainer.setDataDirectory("/content/cars2")
model_trainer.trainModel(num_objects=2, 
                         num_experiments=100, 
                         save_full_model=True, 
                         transfer_from_model="/content/drive/My Drive/Colab Notebooks/models/model_ex-027_acc-0.992647.h5", 
                         enhance_data=True, 
                         batch_size=32,
                         show_network_summary=True)

107/248 [===========>..................] - ETA: 11s - loss: 0.0419 - acc: 0.9965
Epoch 00066: val_acc did not improve from 0.99679
248/248 [==============================] - 115s 465ms/step - loss: 4.9885e-04 - acc: 1.0000 - val_loss: 0.0419 - val_acc: 0.9965
Epoch 67/100
107/248 [===========>..................] - ETA: 11s - loss: 0.0410 - acc: 0.9965
Epoch 00067: val_acc did not improve from 0.99679
248/248 [==============================] - 115s 463ms/step - loss: 7.6490e-04 - acc: 0.9999 - val_loss: 0.0410 - val_acc: 0.9965
Epoch 68/100
107/248 [===========>..................] - ETA: 11s - loss: 0.0415 - acc: 0.9965
Epoch 00068: val_acc did not improve from 0.99679
248/248 [==============================] - 115s 462ms/step - loss: 6.0019e-04 - acc: 0.9999 - val_loss: 0.0415 - val_acc: 0.9965
Epoch 69/100
107/248 [===========>..................] - ETA: 10s - loss: 0.0416 - acc: 0.9965
Epoch 00069: val_acc did not improve from 0.99679
248/248 [==============================] - 113s 45

In [0]:

path = None
img = None
os.mkdir('/content/cars/clips')
main = '/content/cars/test/vehicle/'
for img in os.listdir(main):
    try:        
        output_image = '/content/cars/clips/' + 'out_' + img
        path = main + img

        print("Detecting vehicle for " + img)

        custom_objects = detector.CustomObjects(car=True)
        detections, objects_path = detector.detectCustomObjectsFromImage(custom_objects=custom_objects,
                                                                        input_image=path,
                                                                        extract_detected_objects=True,
                                                                        output_image_path=output_image,
                                                                        minimum_percentage_probability=50)

        for eachObject, eachObjectPath in zip(detections, objects_path):
            print(eachObject['name'] + str(" - ") + str(eachObject["box_points"]) + " - " + str(eachObjectPath))

    except Exception as e:
        print("Exception while predicting: " + str(e))
